# E‑Commerce Data Engineering Lab 


## Step 1 — Hello, Data!
Load a raw CSV and show the first 3 rows.



In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('./data/5000 Sales Records.csv')

df.head(3)


,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Central America and the Caribbean,Antigua and Barbuda,Baby Food,Online,M,12/20/2013,957081544,1/11/2014,552,255.28,159.42,140914.56,87999.84,52914.72
1,Central America and the Caribbean,Panama,Snacks,Offline,C,7/5/2010,301644504,7/26/2010,2167,152.58,97.44,330640.86,211152.48,119488.38
2,Europe,Czech Republic,Beverages,Offline,C,9/12/2011,478051030,9/29/2011,4778,47.45,31.79,226716.10,151892.62,74823.48


## Step 2 — Pick the Right Container

Dict is flexible, lets you map field names to values, and is easy to update or extend. Namedtuple is lightweight and immutable, but harder to modify if new fields appear. Set only stores unique values without order or key–value mapping.I chose dict because it best fits tabular data with field names and allows easy merging, cleaning, and JSON export.

## Step 3 — Implement functions data structure


In [17]:
import pandas as pd

class Record:
    def __init__(self):
        self.records = []
        
    def total(self, path):
        df = pd.read_csv(path)
        self.records = df.to_dict(orient="records")
        return df
        
ri = Record()
ri.total('./data/5000 Sales Records.csv')

print(df.head())            
print(ri.records[2])


   
    


                              Region               Country  Item Type  \
0  Central America and the Caribbean  Antigua and Barbuda   Baby Food   
1  Central America and the Caribbean                Panama     Snacks   
2                             Europe        Czech Republic  Beverages   
3                               Asia           North Korea     Cereal   
4                               Asia             Sri Lanka     Snacks   

  Sales Channel Order Priority  Order Date   Order ID  Ship Date  Units Sold  \
0        Online              M  12/20/2013  957081544  1/11/2014         552   
1       Offline              C    7/5/2010  301644504  7/26/2010        2167   
2       Offline              C   9/12/2011  478051030  9/29/2011        4778   
3       Offline              L   5/13/2010  892599952  6/15/2010        9016   
4       Offline              C   7/20/2015  571902596  7/27/2015        7542   

   Unit Price  Unit Cost  Total Revenue  Total Cost  Total Profit  
0      255.2

## Step 4 — Bulk Loader

Dictionary of Rows

In [7]:
rows_dict = df.to_dict(orient="records")
print(ri.records[0])


{'Region': 'Central America and the Caribbean', 'Country': 'Antigua and Barbuda ', 'Item Type': 'Baby Food', 'Sales Channel': 'Online', 'Order Priority': 'M', 'Order Date': '12/20/2013', 'Order ID': 957081544, 'Ship Date': '1/11/2014', 'Units Sold': 552, 'Unit Price': 255.28, 'Unit Cost': 159.42, 'Total Revenue': 140914.56, 'Total Cost': 87999.84, 'Total Profit': 52914.72}


Dictionary of Columns

In [11]:
colmn_dict = df.to_dict(orient="list")
print(colmn_dict["Region"][:5])

['Central America and the Caribbean', 'Central America and the Caribbean', 'Europe', 'Asia', 'Asia']


## Step 5 — Quick Profiling

In [12]:
min_price = df["Unit Price"].min()
mean_price = df["Unit Price"].mean()
max_price = df["Unit Price"].max()

unique_countries = set(df["Country"])
unique_count = len(unique_countries)

print("Min Unit Price:", min_price)
print("Mean Unit Price:", mean_price)
print("Max Unit Price:", max_price)
print("Unique Country Count:", unique_count)

Min Unit Price: 9.33
Mean Unit Price: 265.74556399999994
Max Unit Price: 668.27
Unique Country Count: 185


## Step 6 — Spot the Grime

## Step 7 — Cleaning Rules

In [15]:
import pandas as pd

df = pd.read_csv("./data/5000 Sales Records.csv")

def clean(df):
    before = df.shape
    
    df["Order Date"] = pd.to_datetime(df["Order Date"], errors="coerce")
    df["Ship Date"] = pd.to_datetime(df["Ship Date"], errors="coerce")

    for col in ["Region", "Country", "Item Type", "Sales Channel", "Order Priority"]:
        df[col] = df[col].astype(str).str.strip().str.title()

    df = df[(df["Units Sold"] > 0) & (df["Unit Price"] > 0) & (df["Unit Cost"] > 0)]
    
    df["Total Profit"] = df["Total Revenue"] - df["Total Cost"]
    
    after = df.shape
    print("Before:", before, "After:", after)
    return df

cleaned = clean(df)


Before: (5000, 14) After: (5000, 14)


## Step 8 — Transformations

In [18]:
import pandas as pd
import numpy as np

# Load your CSV
df = pd.read_csv("./data/5000 Sales Records.csv")

# Define possible coupon codes
codes = ["DISC10", "SAVE20", "HALF", "NONE"]

# Randomly assign coupon codes to each row
df["coupon_code"] = np.random.choice(codes, size=len(df))

print(df[["Country", "Item Type", "Sales Channel", "coupon_code"]].head())


                Country  Item Type Sales Channel coupon_code
0  Antigua and Barbuda   Baby Food        Online        NONE
1                Panama     Snacks       Offline        NONE
2        Czech Republic  Beverages       Offline        NONE
3           North Korea     Cereal       Offline        HALF
4             Sri Lanka     Snacks       Offline        NONE


In [19]:
import numpy as np
import pandas as pd

def parse_coupon(code):
    if pd.isna(code): 
        return 0
    code = str(code).upper().strip()

    # Case 1:
    digits = "".join([c for c in code if c.isdigit()])
    if digits:
        return int(digits)

    # Case 2: 
    if code == "HALF":
        return 50
    if code == "NONE":
        return 0

    # Case 3:
    return 0


codes = ["DISC10", "SAVE20", "HALF", "NONE", "XTR5", None]
df["coupon_code"] = np.random.choice(codes, size=len(df))

df["Discount (%)"] = df["coupon_code"].apply(parse_coupon)

df["Final Price"] = df["Unit Price"] * df["Units Sold"] * (1 - df["Discount (%)"]/100)

print(df[["coupon_code", "Discount (%)", "Final Price"]].head())


  coupon_code  Discount (%)  Final Price
0        XTR5             5   133868.832
1        XTR5             5   314108.817
2      SAVE20            20   181372.880
3        None             0  1854591.200
4        HALF            50   575379.180


## Step 9 — Feature Engineering

In [20]:
import pandas as pd
from datetime import datetime

df = pd.read_csv('./data/5000 Sales Records.csv')

df["Order Date"] = pd.to_datetime(df["Order Date"], errors = "coerce")

today = pd.Timestamp.today().normalize()
df["days_since_purchase"] = (today - df["Order Date"]).dt.days

print(df[["Order Date", "days_since_purchase"]].head())

  Order Date  days_since_purchase
0 2013-12-20                 4301
1 2010-07-05                 5565
2 2011-09-12                 5131
3 2010-05-13                 5618
4 2015-07-20                 3724


## Step 10 — Mini-Aggregation

In [21]:
df = pd.read_csv('./data/5000 Sales Records.csv')

revenue_country = df.groupby(["Country", "Total Revenue"]).agg(unit_cost=("Unit Cost", "mean"), total_units=("Units Sold", "count"), total_cost=("Total Cost", "sum"))
print(revenue_country)

                           unit_cost  total_units  total_cost
Country     Total Revenue                                    
Afghanistan 13705.77            6.92            1    10165.48
            79799.34           97.44            1    50961.12
            225574.80          56.67            1   156409.20
            246958.18          90.93            1   145760.79
            274285.88          56.67            1   190184.52
...                              ...          ...         ...
Zimbabwe    1216409.20        159.42            1   759636.30
            1486216.82         90.93            1   877201.71
            3205942.11        364.69            1  2771279.31
            3589946.44        502.54            1  2699644.88
            5298244.56        524.96            1  4271074.56

[5000 rows x 3 columns]


## Step 11 — Serialization Checkpoint

In [22]:
import pandas as pd

df = pd.read_csv('./data/5000 Sales Records.csv')

df.to_json("cleaned_sales.json", orient="records", indent=4, date_format="iso")

print("Saved cleaned_sales.json")

Saved cleaned_sales.json


## Step 12 — Soft Interview Reflection (<120 words)

Functions help breakdown long complicated codes into smaller reusable steps. The idea is to put some commonly or repeatedly done task together and make a function so that instead of writing the same code again and again for different inputs, we can do the function calls to reuse code contained in it over and over again. This makes the code cleaner and easy to understand. They also make testing easier. The function can be reused over and over agin even in different files too. Overall they help reduce errors, make the work more organized, reliable and efficient.